In [20]:
from scipy.io import loadmat
import meshplot as mp
import numpy as np
from pathlib import Path
import open3d as o3d #install with conda
from tqdm import tqdm
from multiprocessing import Pool
import h5py

In [21]:
data_ids = loadmat('VertexIdxSpecParts.mat')
hands = data_ids['idxHand']-1
data = loadmat('faceShapeModel.mat')
f = data['faces'] - 1
not_hands = [i for i in range(f.max()+1) if i not in hands]
mask = np.all(np.isin(f,not_hands),axis=1)
f = f[mask]
source_folder = Path('caesar-fitted-meshes')
target_folder = Path('../dataset/scans/')

In [23]:
def convert(source_path):
    #print(source_path)
    if not source_path.suffix == '.mat': 
        return
    #print(source_path)
    filename = source_path.relative_to(source_folder)
    target_subfolder = target_folder / filename.with_suffix('')
    target_subfolder = target_subfolder.absolute()
    #print(target_subfolder)
    target_subfolder.mkdir(parents=False, exist_ok=True)
    
    dest = target_subfolder/filename.with_suffix('.xyz')
    if dest.is_file():
        return
    data = loadmat(source_path)
    v = data['points'] / 1000
    vmesh = o3d.utility.Vector3dVector(v)
    fmesh = o3d.utility.Vector3iVector(f)
    mesh = o3d.geometry.TriangleMesh(vmesh,fmesh)
    mesh = mesh.compute_vertex_normals()
    
    cluster_idx, cluster_size, _ = [np.asarray(x) for x in mesh.cluster_connected_triangles()]
    tri_mask = cluster_idx != np.argmax(cluster_size)
    mesh.remove_triangles_by_mask(tri_mask)
    
    o3dCloud = mesh.sample_points_poisson_disk(20000)
    
    cloudv = np.asarray(o3dCloud.points)
    cloudn = np.asarray(o3dCloud.normals)
    cloudc = np.hstack([cloudv,cloudn])
    
    np.savetxt(str(dest), cloudc, delimiter=' ',
              fmt='%f')
    #mp.plot(cloudc[:,:3])
    #mp.plot(v,f)
    #break


In [24]:
pathlist = list(source_folder.iterdir())

with Pool(processes=20) as p:
    with tqdm(total=len(pathlist)) as pbar:
        for i, _ in enumerate(p.imap_unordered(convert, pathlist)):
            pbar.update()
#for source_path in tqdm(list(source_folder.iterdir())):
    

100%|██████████| 4309/4309 [00:00<00:00, 12204.34it/s]


In [28]:
for x in pathlist:
    if 'nl_578' in str(x):
        print(x)

caesar-fitted-meshes/nl_5785a.mat
caesar-fitted-meshes/nl_5782a.mat
caesar-fitted-meshes/nl_5780a.mat
caesar-fitted-meshes/nl_5786a.mat
caesar-fitted-meshes/nl_5784a.mat
caesar-fitted-meshes/nl_5783a.mat


In [41]:
verts = None
with h5py.File('dfaust/registrations_m.hdf5') as g:
    keys = list(g.keys())
    sorted(keys)
    print(keys)
    f = g['faces'].value.copy()
    verts = g['50002_chicken_wings'].value.copy()
    print(verts.shape)
mp.plot(verts[:,:,0],f)

['50002_chicken_wings', '50002_hips', '50002_jiggle_on_toes', '50002_jumping_jacks', '50002_knees', '50002_light_hopping_loose', '50002_light_hopping_stiff', '50002_one_leg_jump', '50002_one_leg_loose', '50002_punching', '50002_running_on_spot', '50002_shake_arms', '50002_shake_hips', '50002_shake_shoulders', '50007_chicken_wings', '50007_jiggle_on_toes', '50007_jumping_jacks', '50007_knees', '50007_light_hopping_loose', '50007_light_hopping_stiff', '50007_one_leg_jump', '50007_one_leg_loose', '50007_punching', '50007_running_on_spot', '50007_shake_arms', '50007_shake_hips', '50007_shake_shoulders', '50009_chicken_wings', '50009_hips', '50009_jiggle_on_toes', '50009_jumping_jacks', '50009_light_hopping_loose', '50009_light_hopping_stiff', '50009_one_leg_jump', '50009_one_leg_loose', '50009_punching', '50009_running_on_spot', '50009_shake_hips', '50026_chicken_wings', '50026_hips', '50026_jiggle_on_toes', '50026_jumping_jacks', '50026_knees', '50026_light_hopping_loose', '50026_light_ho

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0263963…

In [6]:

def convert_dfaust(params):
    person_name, scan_name, v  = params
        #print(source_path)
        
    #print(source_path)
    filename = Path(scan_name)
    target_subfolder = target_folder / person_name
    target_subfolder = target_subfolder.absolute()
    #print(target_subfolder)
    target_subfolder.mkdir(parents=False, exist_ok=True)
    if (target_subfolder/filename).is_file():
        return

    #v = data['points'] / 1000
    vmesh = o3d.utility.Vector3dVector(v)
    fmesh = o3d.utility.Vector3iVector(f)
    mesh = o3d.geometry.TriangleMesh(vmesh,fmesh)
    mesh = mesh.compute_vertex_normals()
    
    cluster_idx, cluster_size, _ = [np.asarray(x) for x in mesh.cluster_connected_triangles()]
    tri_mask = cluster_idx != np.argmax(cluster_size)
    mesh.remove_triangles_by_mask(tri_mask)
    
    o3dCloud = mesh.sample_points_poisson_disk(20000)
    
    cloudv = np.asarray(o3dCloud.points)
    cloudn = np.asarray(o3dCloud.normals)
    cloudc = np.hstack([cloudv,cloudn])
    
    np.savetxt(str(target_subfolder/filename), cloudc, delimiter=' ',
              fmt='%f')

In [7]:
convert_dfaust(("50002","chicken_wings.000000.xyz",verts[:,:,0]))

In [8]:
#param_list = [("50002","chicken_wings.%06d.xyz" % i,verts[:,:,i].copy()) for i in range(verts.shape[2])]


In [29]:

with Pool(processes=20) as p:
    with tqdm(total=len(param_list)) as pbar:
        for i, _ in enumerate(p.imap_unordered(convert_dfaust, param_list)):
            pbar.update()

100%|██████████| 216/216 [01:45<00:00,  3.08it/s]


In [18]:
#verts = None
mapper = None
import json
with open('dfaust_registration_to_scan_ids.json','r') as g:
    mapper = json.load(g)
param_list = []
for sex in ['m','f']:
    with h5py.File('dfaust/registrations_%s.hdf5' % sex) as g:
        f = g['faces'].value.copy()
        #print(g.keys())
        for k in g.keys():
            if k == 'faces':
                continue

            pid,sid = k.split('_')[0],"_".join(k.split('_')[1:])
            verts = g[k].value.copy()




            for i in range (verts.shape[2]):
                j = mapper[k][i]
                param_list.append((pid,"%s.%06d.xyz" % (sid,j),verts[:,:,i]))
            #print(verts.shape)

In [19]:
with Pool(processes=20) as p:
    with tqdm(total=len(param_list)) as pbar:
        for i, _ in enumerate(p.imap_unordered(convert_dfaust, param_list)):
            pbar.update()

100%|██████████| 41220/41220 [00:26<00:00, 1540.74it/s]


In [16]:
#create idx list
mapper = {}
gtfolder = Path('/media/caduser/data/projects/hyperdocker/hypermod/dataset/scans')
for sex in ['m','f']:
    with h5py.File('dfaust/registrations_%s.hdf5' % sex) as g:
        for k in g.keys():
            if k == 'faces':
                continue
            size = g[k].value.shape[2]
            pid,sid = k.split('_')[0],"_".join(k.split('_')[1:])

            person_folder = gtfolder/pid
            idxmin = 99999999
            idxmax = -1
            idxcount = 0
            idxlist = []
            for filename in person_folder.iterdir():
                split = filename.with_suffix('').name.split('.')
                if len(split) != 2:
                    print(filename)
                fsid,fid = split
                if fsid != sid:
                    continue
                idxcount += 1
                if idxmin > int(fid):
                    idxmin = int(fid)
                if idxmax < int(fid):
                    idxmax = int(fid)
                idxlist.append(int(fid))
            idxlist = sorted(idxlist)
            mapper[k] = idxlist
            assert(idxcount == size)

In [17]:
import json
with open('dfaust_registration_to_scan_ids.json','w') as g:
    json.dump(mapper,g)